<a href="https://colab.research.google.com/github/Baah134/Baah134/blob/main/Whisper/Whisper_Transcription_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
import os
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 📂 Paths
AUDIO_DIR = '/content/drive/MyDrive/DeepLearning/Whisper/dev'  # Folder containing subfolders of audio files
CSV_PATH = "/content/drive/MyDrive/DeepLearning/Whisper/transcripts.csv"

In [ ]:
import os
import pandas as pd
import random

# 📂 Paths
AUDIO_DIR = '/content/drive/MyDrive/DeepLearning/Whisper/dev'  # Folder containing subfolders of audio files
CSV_PATH = "/content/drive/MyDrive/DeepLearning/Whisper/transcripts.csv"

# 📌 Load transcription CSV
df_transcriptions = pd.read_csv(CSV_PATH)

# Ensure the CSV has correct column names
AUDIO_COLUMN = "audio_path"  # Column name containing full paths in the CSV
TEXT_COLUMN = "transcript"  # Column name for transcriptions

# Step 1️⃣: Extract just the filename (e.g., "audio_001.wav") from the CSV audio paths
df_transcriptions["filename"] = df_transcriptions[AUDIO_COLUMN].apply(lambda x: os.path.basename(x))

# Convert CSV into a dictionary {filename: transcription}
transcription_dict = dict(zip(df_transcriptions["filename"], df_transcriptions[TEXT_COLUMN]))

# Step 2️⃣: Get list of all .wav files inside subdirectories
all_audio_files = []
for root, dirs, files in os.walk(AUDIO_DIR):
    for file in files:
        if file.endswith(".wav"):
            full_path = os.path.join(root, file)
            all_audio_files.append(full_path)

# Step 3️⃣: Extract filenames from actual audio file paths
audio_dict = {os.path.basename(file): file for file in all_audio_files}  # {filename: full_path}

# Step 4️⃣: Match filenames in CSV with actual audio file paths
matched_audio_files = {audio_dict[f]: transcription_dict[f] for f in transcription_dict if f in audio_dict}

# Step 5️⃣: Pick 100 random samples
random.seed(42)
selected_samples = random.sample(list(matched_audio_files.items()), 100)

# 🎯 Print first 5 matched samples
for audio_path, transcription in selected_samples[:5]:
    print(f"Audio File: {audio_path}")
    print(f"Transcription: {transcription}")
    print("-" * 50)


In [ ]:
!pip install openai-whisper jiwer pandas tqdm


import whisper
import pandas as pd
import os
from tqdm import tqdm
from jiwer import wer

# Load Whisper model (small for speed, you can use "medium" or "large" for better accuracy)
model = whisper.load_model("medium")

# CSV Output Path
CSV_OUTPUT = "transcription_results.csv"

# Store results
results = []

# Loop through the selected 100 audio samples
for audio_path, actual_transcription in tqdm(selected_samples, desc="Processing Audio"):

    # Transcribe audio using Whisper
    transcription_result = model.transcribe(audio_path)
    whisper_transcription = transcription_result["text"]

    # Calculate Word Error Rate (WER)
    error_rate = wer(actual_transcription, whisper_transcription)

    # Store result
    results.append({
        "Audio File": audio_path,
        "Actual Transcription": actual_transcription,
        "Whisper Transcription": whisper_transcription,
        "Word Error Rate": error_rate
    })

# Convert results to DataFrame
df_results = pd.DataFrame(results)

# Save to CSV
df_results.to_csv(CSV_OUTPUT, index=False)

print(f"Transcriptions saved to {CSV_OUTPUT}")
